In [ ]:
#provide the following environment variables to test from your end. Additioally ngrok token is required for making the notebook publicly accessible

import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"] = ""
os.environ["ARANGODB_URL"] = ""
os.environ["ARANGODB_USERNAME"] = ""
os.environ["ARANGODB_PASSWORD"] = ""
os.environ["DB_NAME"] = ""

In [ ]:
#ngrok authentication token
!ngrok authtoken ""

In [ ]:
!pip install nx-arangodb

In [ ]:
!nvidia-smi -L 

In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
!pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com

In [ ]:
!pip install --upgrade langchain langchain-community langchain-openai langgraph

In [ ]:
!pip install -U langchain-anthropic

In [ ]:
!pip install arango_datasets

In [ ]:
%env NX_CURGAPH_AUTOCONFIG=True
import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient
from IPython import get_ipython

import datetime
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re
import sys
import collections

from IPython.display import display, Markdown, Latex
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model='claude-3-7-sonnet-20250219')

env: NX_CURGAPH_AUTOCONFIG=True


[02:53:29 +0000] [INFO]: NetworkX-cuGraph is available.


In [ ]:
#Creating a DB instance
arangodb_url = os.getenv("ARANGODB_URL")
arangodb_username = os.getenv("ARANGODB_USERNAME")
arangodb_password = os.getenv("ARANGODB_PASSWORD")
db_name = os.getenv("DB_NAME")

db = ArangoClient(hosts=arangodb_url).db(db_name, username=arangodb_username, password=arangodb_password, verify=True)
print(db)

<StandardDatabase arangoDB_hackathon>


In [ ]:
#Datasets info
from arango_datasets import Datasets

datasets = Datasets(db)

#CVE data set info
print(datasets.dataset_info("CVE"))

# Also, from the given dataset, we noticed product_vendor relationships are broken as all the records have same product ID in _from field.
# We resolved that by creating a new json file based on the CVE CSV files

Output()

{'files': 6, 'edges': [{'collection_name': 'product_cve', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/cve/edge_collection/product_cve.json']}, {'collection_name': 'product_vendor', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/cve/edge_collection/product_vendor.json']}, {'collection_name': 'vendor_cve', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/cve/edge_collection/vendor_cve.json']}], 'vertices': [{'collection_name': 'cve', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/cve/vertex_collection/cve.json']}, {'collection_name': 'product', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/cve/vertex_collection/product.json']}, {'collection_name': 'vendor', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/cve/vertex_collection/vendor.json']}], 'file_type': 'json', 'label': 'CVE', 'edge_definitions': [{'edge_collection': 'product_cve', 'from_vertex_collections': ['product'], 'to_vertex_co

In [ ]:
#Loading data into ArangoDB uncomment the below line to load data into ArangoDB
#datasets.load("CVE")

In [ ]:
arango_graph = ArangoGraph(db)

In [ ]:
#Creating a view for performing ArangoSearch(full text search)

views = db.views()
view = None
for v in views:
    if v['name'] == 'cve_search':
        view = db.view('cve_search')
view_link_info = None
if(view):
    view_link_info = view
else:
    link = { 
      'includeAllFields': True,
      'fields' : {
        'access_complexity': {'analyzers': ['identity']},
        'cwe_name': {'analyzers': ['identity']},
        'impact_confidentiality': {'analyzers': ['identity']},
        'summary': {'analyzers': ['identity']},
        'cwe_code': {'analyzers': ['identity']},
        'cvss': {'analyzers': ['identity']},
        'mod_date': {'analyzers': ['identity']},
        'pub_date': {'analyzers': ['identity']},
        'impact_integrity': {'analyzers': ['identity']},
        'access_vector': {'analyzers': ['identity']},
        'access_authentication': {'analyzers': ['identity']},
        'impact_availability': {'analyzers': ['identity']},
      },
    };
    
    view = db.create_view(
        name='cve_search',
        view_type='arangosearch',
        properties = { 
            'links': { 'cve': link }
        }
    )
    view_link_info = view

print(view_link_info)

{'global_id': 'h59F07127C7DF/4915309891', 'id': '4915309891', 'name': 'cve_search', 'type': 'arangosearch', 'cleanup_interval_step': 2, 'commit_interval_msec': 1000, 'consolidation_interval_msec': 1000, 'consolidation_policy': {'type': 'tier', 'segments_min': 1, 'segments_max': 10, 'segments_bytes_max': 5368709120, 'segments_bytes_floor': 2097152, 'min_score': 0}, 'primary_sort': [], 'primary_sort_compression': 'lz4', 'stored_values': [], 'writebuffer_idle': 64, 'writebuffer_active': 0, 'writebuffer_max_size': 33554432, 'links': {'cve': {'analyzers': ['identity'], 'fields': {'impact_integrity': {}, 'mod_date': {}, 'access_complexity': {}, 'access_vector': {}, 'impact_confidentiality': {}, 'pub_date': {}, 'access_authentication': {}, 'summary': {}, 'cwe_name': {}, 'impact_availability': {}, 'cwe_code': {}, 'cvss': {}}, 'includeAllFields': True, 'storeValues': 'none', 'trackListPositions': False}}, 'optimizeTopK': []}


In [ ]:
# ArangonSearch tool functionality
from langchain_core.tools import ToolException
from pydantic import BaseModel, Field
from langchain_anthropic import ChatAnthropic
from langchain.tools import BaseTool
import pydantic
from typing import Any
import re
import time
import logging

class ArangoSearchInput(BaseModel):
    query: str = Field(description="Natural language query related to ArangoSearch")

class ArangoSearch(BaseTool):
    name: str = "arangosearch"
    description: str = "A tool for converting natural language queries into AQL queries for ArangoSearch."
    arango_graph: dict = None
    llm: Any = ChatAnthropic(model='claude-3-7-sonnet-20250219')
    max_retries: int = 3
    view_schema: dict = None
    return_search_query : bool = False

    def __init__(self, arango_graph, llm, max_retries=3, view_schema=None, return_search_query=False):
        """Initialize the ArangoSearch tool.
        
        Args:
            arango_graph: An object with a query method to execute AQL queries
            llm: LLM to use for query generation
            max_retries: Maximum number of retries for failed queries
            view_schema: Schema of the ArangoSearch view
            return_search_query: Weather to return the generated query or not
        """
        super().__init__()
        self.arango_graph = arango_graph
        self.llm = ChatAnthropic(model='claude-3-7-sonnet-20250219')
        self.max_retries = max_retries
        self.view_schema = view_schema or {}
        self.return_search_query = return_search_query

    def _generate_query(self, query: str) -> str:
        """Generate an AQL query from a natural language query."""
        schema_info = f"View schema: {self.view_schema}" if self.view_schema else ""
        view_name = self.view_schema['name']
        view_field = self.view_schema['links']['cve']['fields']
        prompt = f"""
        Convert the following natural language query {query} into an ArangoSearch AQL query using the info from {schema_info} and the name of the view is 
        {view_name}
        The query should be valid AQL and compatible with the ArangoDB Python driver.
        Do not include Python code for execution, just return the AQL query as a valid string.
        Ensure the query uses the correct collections and fields from {schema_info}.
        Only return the useful fields instead of the whole doc unless explicitly asked for it.
        """
        response = self.llm.invoke(prompt)
        query_text = response.content.strip()

        # Extract AQL query from response using regex
        match = re.search(r"```(?:aql)?\n(.*?)```", query_text, re.DOTALL)
        return match.group(1).strip() if match else query_text

    def _execute_query(self, aql_query: str) -> Any:
        try:
            return self.arango_graph.query(aql_query)
        except Exception as e:
            raise ToolException(f"Query execution failed: {str(e)}")

    def _format_results(self, search_query: Any, results: Any) -> dict:
        formatted_results = {
            "results": results
        }
        if self.return_search_query:
            formatted_results['search_query'] = search_query
        return formatted_results


    def _run(self, query: str) -> str:
        retries = 0
        last_error = None

        while retries < self.max_retries:
            try:
                print("Generating query")
                aql_query = self._generate_query(query)
                print("ArangoSearch Query:",aql_query)
                
                # Execute the query
                results = self._execute_query(aql_query)
                # Format and return the results
                return self._format_results(aql_query, results)

            except ToolException as e:
                last_error = str(e)
                retries += 1

                if retries < self.max_retries:
                    error_prompt = f"""
                    The following AQL query failed with error: {last_error}
                    Failed query: {aql_query}
                    Please fix the query to address this error. The original natural language request was:
                    "{query}"
                    View schema: {self.view_schema}
                    """

                    response = self.llm.invoke(error_prompt)
                    new_query_text = response.content.strip()

                    # Extract AQL query from response using regex
                    match = re.search(r"```(?:aql)?\n(.*?)```", new_query_text, re.DOTALL)
                    aql_query = match.group(1).strip() if match else new_query_text

                    # Brief delay before retrying
                    time.sleep(1)
                else:
                    return f"Failed to execute query after {self.max_retries} attempts. Last error: {last_error}"

        return f"Failed to execute query. Error: {last_error}"

In [ ]:
#Graph instance
graph_name = "CVE"
G_adb = nxadb.Graph(name=graph_name, db=db)

[02:54:00 +0000] [INFO]: Graph 'CVE' exists.
[02:54:00 +0000] [INFO]: Default node type set to 'cve'


In [ ]:
#Setting nx backend and configuring the arangodb to use GPU
nx.config.backend_priority = ["cugraph"]
nx.config.backends.arangodb.use_gpu = True

In [ ]:
#Verifying the configs
import networkx as nx

print("Backend Priority:", nx.config.backend_priority)
print("Using cuGraph:", "cugraph" in nx.config.backend_priority.algos)

Backend Priority: BackendPriorities(algos=[], generators=[])
Using cuGraph: False


In [ ]:
#CVE Graph Schema
print(arango_graph.schema)

{'Graph Schema': [{'graph_name': 'CVE', 'edge_definitions': [{'edge_collection': 'product_cve', 'from_vertex_collections': ['product'], 'to_vertex_collections': ['cve']}, {'edge_collection': 'product_vendor', 'from_vertex_collections': ['product'], 'to_vertex_collections': ['vendor']}, {'edge_collection': 'vendor_cve', 'from_vertex_collections': ['vendor'], 'to_vertex_collections': ['cve']}]}], 'Collection Schema': [{'collection_name': 'cve', 'collection_type': 'document', 'document_properties': [{'name': '_key', 'type': 'str'}, {'name': '_id', 'type': 'str'}, {'name': '_rev', 'type': 'str'}, {'name': 'mod_date', 'type': 'str'}, {'name': 'pub_date', 'type': 'str'}, {'name': 'cvss', 'type': 'float'}, {'name': 'cwe_code', 'type': 'int'}, {'name': 'cwe_name', 'type': 'str'}, {'name': 'summary', 'type': 'str'}, {'name': 'access_authentication', 'type': 'str'}, {'name': 'access_complexity', 'type': 'str'}, {'name': 'access_vector', 'type': 'str'}, {'name': 'impact_availability', 'type': 'st

In [ ]:
# Graph statistics

graph_stats = f"""
    Graph Statistics:
    - Number of nodes: {G_adb.number_of_nodes()}
    - Number of edges: {G_adb.number_of_edges()}
    """
print(graph_stats)


    Graph Statistics:
    - Number of nodes: 145506
    - Number of edges: 316954
    


In [ ]:
#Tools Explanation
"""
Every tool returns two types of data 
results: These are sent to LLM which summarize it and creates a markdown
artifacts: artifacts are shared with the claude MCP for creating stunning visualizations. These won't be passed through LLM thus saving tokens and retaining the format

artifacts use-case:
When text_to_nx_algorithm_to_text generates networkX code this is used for displaying in claude artifact. However, this is not helpful for LLM itself when summarizing the results.

Artifacts for the below tools contain:
executed queries, query results and NLP(Natural language) query
"""

In [ ]:
@tool(response_format="content_and_artifact")
def text_to_aql_arangosearch(query: str):
    """This tool is available to invoke to interact with arangodb directly to perform a 
    full text search based on the user natural query by converting it to AQL supported full text search query, execute the query and
    translate result to natural language.
    """
    print("Using ArangoSearch Tool")
    print("Query:", query)
    arango_tool = ArangoSearch(
        arango_graph= ArangoGraph(db),
        llm= ChatAnthropic(model='claude-3-7-sonnet-20250219'),
        view_schema=view_link_info,
        return_search_query=True
    )
    result = arango_tool.run(query)
    artifact = {
        'result' : result['results'],
        'query' : query,
        'search_query' : result['search_query']
    }
    return result['results'], artifact

In [ ]:
@tool(response_format="content_and_artifact")
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """
    print("Using Arango Query Tool")
    print("Query:", query)
    llm = ChatAnthropic(model='claude-3-7-sonnet-20250219')

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
        allow_dangerous_requests=True,
        return_aql_query=True,
        return_aql_result=True
    )
    
    result = chain.invoke(query)
    artifact = {
        'query' : query,
        'aql_query' : result['aql_query'],
        'aql_result' : result['aql_result']
    }
    return str(result["result"]), artifact

In [ ]:
#Tool text_to_nx_algorithm_to_text (networkX code generation)
"""
This tool utilizes 3 different types of prompts to handle code generation, optimization and corrections.

Initially, it creates a detailed prompt using the provided instructions, schema, query and graph_stats.
Then, the generated code is optimized in the next step. To make sure the time constraints and sampling are in place.
Next the code execution takes place, on success it'll return the info to the LLM to summarize

In case of failure/error, all the initial details (code, error, initial_prompt) are used to create a code fix prompt and pass to LLM, for multiple attempts it'll keep track of the 
previous errors as well.
"""

In [ ]:
@tool(response_format="content_and_artifact")
def text_to_nx_algorithm_to_text(query):
    """This tool invokes a NetworkX Algorithm on the ArangoDB Graph."""
    
    print("Using networkX code generation tool")
    print("\nGenerating NetworkX code")
    print("Query:", query)
    FINAL_RESULT = "There is some issue when generating/executing code"
    attempt = 0
    MAX_ATTEMPTS = 3
    llm = ChatAnthropic(
        model='claude-3-7-sonnet-20250219',
        temperature=0,
        max_tokens=7200,
        timeout=None,
    )
    
    graph_stats = f"""
    Graph Statistics:
    - Number of nodes: {G_adb.number_of_nodes()}
    - Number of edges: {G_adb.number_of_edges()}
    """

    networkX_code_generation_prompt = f"""
    {datetime.datetime.now(datetime.timezone.utc)}
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}
    I have the following graph analysis query: {query}.
    Additional graph context: {graph_stats}
    
    Generate optimized Python code using NetworkX to answer the query. Your code must execute within 60 seconds:

    1. Smart Algorithm & Library Selection (Before Code Generation)*  
    •  *Analyze the graph properties* (size, density, directed/undirected, weighted/unweighted) and *determine the best algorithm upfront* before generating code.  
    •  *Use cuGraph for large-scale graphs when possible* to maximize performance and GPU acceleration.  
    •  *Automatically apply built-in optimizations* available in the selected library for the best execution speed.  
       *Import the needed graph algorithms from netwrokX package only*
       *Check for deprecated functions and avoid outdated syntax*
        
    2. ALWAYS include performance optimization parameters:
       - For betweenness_centrality: Use k=min(100, number_of_nodes) for approximation
       - For eigenvector_centrality: Set max_iter=100 and tol=1e-4
       - For PageRank: Set max_iter=100 and tol=1e-6
       - For any clustering algorithms: Use appropriate sampling or approximation
       - For any expensive networkX  : Include a timeout mechanism (max 60 seconds). Write the program in a way it won't take more than 60 seconds.
       - Also, remember we have Nvidia GPU access and networkX is configured with cugraph backend which take cares of execution.
       - It is important to write optimized code to keep the execution within 60 seconds, this is a hard limit so apply this to every program. 
           In case this exceeds, throw a error saying taking too much time. while taking advantage of networkX cu-graph backend.
       - Always make sure to write the code in a way it won't take more than a minute .i.e., by setting sampling to less for time sensitive algorithms.

    3. Performance Optimization & Execution Constraints  
    •  *Ensure the generated code executes within a reasonable time limit* and does not get stuck in infinite loops.  
    •  *Use efficient graph traversal and search techniques* (e.g., approximate methods, parallelization, sampling) when handling large datasets.  
    •  *Do not generate code that runs indefinitely or consumes excessive memory.*
       *Try to keep print statements for major operations, so it'll be easy to track code execution.
    
    4. If you're performing any operation related to diGraph, convert the graph to directed graph (diGraph) if it is not a diGraph.
    
    5. Define a variable `FINAL_RESULT` containing the answer in human-readable format (dict, list, string, or number) Make sure you're using names rather than their Id's refer the 
    schema to get the required fields. **Always limit dict or list items to 50**
    
    6. Include error handling with try/except blocks for all major operations. Also for final results don't save the error in final result throw the error as expection,
    so the code can be generated again using that error as reference.
    
    7. For large graphs (>1000 nodes), use appropriate sampling techniques:
       - Sample nodes when computing centrality measures
       - Limit path computations
       - Use approximation algorithms when exact solutions are too expensive
    
    8. Include clear comments explaining the algorithm choice and each major step
    
    Your code must be complete, functional, and optimized to run in under 60 seconds even on large graphs.
    Only provide executable Python code without any additional instructions or markdown.
    """
    
    text_to_nx = llm.invoke(networkX_code_generation_prompt).content
    
    # Clean up the response to extract only the Python code
    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    networkX_code_optimization_prompt = f"""
    Optimize the networkX code
    {text_to_nx_cleaned}  so it won't take more than a min, also we have configured nx-cugraph as backend to networkX.
    Also, you should only return the python code and nothing else as you're running in an unsupervised evironment.
    This code was generated by llm using the prompt
    {networkX_code_generation_prompt}
    """

    print("\nOptimizing networkX code")
    text_to_nx = llm.invoke(networkX_code_generation_prompt).content
    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    with open("networkX.py", "w") as f:
        f.write(text_to_nx_cleaned)
    while attempt < MAX_ATTEMPTS:
        print("\n2) Executing NetworkX code")
        with open("networkX.py", "r") as f:
            code = f.read()
        try:
            ip = get_ipython()
            result = ip.run_cell(code)
            if result.error_before_exec or result.error_in_exec:
                raise Exception(f"Execution error: {result.error_in_exec}")
            
            FINAL_RESULT = ip.user_ns['FINAL_RESULT']
            break
        except Exception as e:
            attempt +=1
            print(f"EXEC ERROR: {e}")
            print("\nRefactoring networkX code to resolve issues. Attempt: ",attempt)
            try:
                error_fix_prompt = f"""
                    Resolve the error "{e}" that occurred while executing the following NetworkX code:
                    {code} using prompt {networkX_code_generation_prompt}
                    *Important guidelines for fixing the code:*
                       - Since code generation is unsupervised, do NOT suggest installing missing packages.
                       - Instead, completely rewrite the code using available alternatives.
                       - The environment is a Jupyter notebook with NetworkX installed (with nx-cugraph backend support).
                       - The goal is to implement NetworkX graph functionalities or algorithms that properly address the original user query.    
                    *Requirements for your response:*
                       - Provide ONLY executable Python code without any explanations, markdown formatting, or additional instructions.
                       - For handling errors: Do not catch and save errors in the final result. Instead, allow exceptions to be thrown so they can serve as references for future code generation.
                       - Follow all instructions from the original prompt.
                       - Test your solution thoroughly to ensure it works with the available libraries.
                       - Ensure all variable names and functions are properly defined.
                       - Include appropriate error handling that follows the guidelines above.
                """
                text_to_nx = llm.invoke(error_fix_prompt).content
                text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
                with open("networkX.py", "w") as f:
                    f.write(text_to_nx_cleaned)
            except Exception as e:
                print(f"EXEC ERROR: {e}")
    artifact = {
        'query' : query,
        'networkX_code': text_to_nx_cleaned,
        'final_result': FINAL_RESULT
    }
    return FINAL_RESULT, artifact

In [ ]:
#Creating an agent
"""
A ReAct agent is created to answer the user queries, this agent has access to multiple tools to perform different types of operations and analysis on the graph data.
The agent supports both single and hybrid calls, based on the user query or explicit directions.
"""

In [ ]:
tools = [text_to_aql_to_text, text_to_nx_algorithm_to_text, text_to_aql_arangosearch]

final_state = None
data = None
systemPrompt = {
        "role":"system",
        "content":f"""
            {datetime.datetime.now(datetime.timezone.utc)}
            You are a helpful agent specializing in graph databases like ArangoDB, AQL (ArangoDB Query Language), ArangoSearch, NetworkX, and cuGraph.
            Your task is to answer user queries by selecting and combining appropriate tools based on the query requirements.

            When deciding which tool(s) to use:
            1. For basic data retrieval or filtering from collections and any sql like questions, use 'text_to_aql_to_text' (AQL queries)
            2. For graph traversal, path finding, centrality, or complex graph algorithms, use 'text_to_nx_algorithm_to_text' (NetworkX)
            3. For full-text search, tokens search, proximity and fuzzy matching within document fields and for any questions that require word search or ArangoSearch functionalities use 'text_to_aql_arangosearch' (ArangoSearch)
            
            Important guidelines:
            - All queries are case-sensitive; don't capitalize anything mentioned in the queries
            - Present all results using markdown formatting for improved readability
            - When a query requires multiple steps, clearly explain your reasoning for each tool selection
            - For complex queries, break down the problem and execute tools in sequence, using output from one tool as input to another
            - If information from one tool is needed to complete another operation, explain the workflow and dependencies
            - When the user asks the data to be in some specific format like graph, heatmap or barchart etc., prepare the data to be usable in that format but do not create any visualization. Also try to make use of networkX if needed.
            
            Example multi-step approach:
            ** when passing details like vendor names or ids from one tool to another pass them as it is don't change anything, don't capitalize or remove underscores as it'll break the
            next step. for example don't change 'debian_linux' to Debian Linux. This should be avoided as it'll break the flow.
            1. Use ArangoSearch to find relevant documents based on search terms
            2. Extract identifiers or relationships from those results
            3. Use AQL to gather additional structured data about these entities
            4. Pass the complete dataset to NetworkX for advanced graph analysis
            5. Use multitool calling only when necessary
            
            Always clearly explain your approach before executing tools and summarize findings afterwards.
            **For simple queries don't break down them into multiple pieces only do that for complex and multi step ones. If something can be achieved as a single query make sure to
            prioritize it.
        """
    }

def query_graph(query):
    llm = ChatAnthropic(
        model='claude-3-7-sonnet-20250219',
        temperature=0
    )
    app = create_react_agent(llm, tools)
    final_state = app.invoke({"messages": [systemPrompt, {"role": "user", "content": query}]})
    messages = final_state["messages"]
    tool_data = filter_messages(messages, include_types="tool")
    tools_used = [msg.name for msg in tool_data]
    is_multi_tool = len(set(tools_used)) > 1
    if tools_used:
        print(f"Tools used: {tools_used}")
        print(f"Multi-tool calling: {is_multi_tool}")
        
    # You can also track the sequence
    tool_sequence = ""
    if is_multi_tool:
        tool_sequence = f"Tool sequence: {' -> '.join(tools_used)}"
        print(tool_sequence)
  
    markup_data = final_state["messages"][-1].content
    display(Markdown(markup_data))
    artifacts = []
    for tool in tool_data:
        if hasattr(tool, "artifact"):
            artifacts.append(tool.artifact)
    
    return {
        'isMultiTool' : is_multi_tool,
        'toolsUsed' : tools_used,
        'artifacts': artifacts,
        'summary': markup_data
    }

In [ ]:
#Example query for Arango Query Language
query_graph("which vendor has more CVE's");

Using Arango Query Tool
Query: Which vendor has the highest number of CVEs? Please provide the vendor name and the count of CVEs.


> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH vendor, vendor_cve
FOR v IN vendor
    LET cve_count = LENGTH(
        FOR vc IN vendor_cve
        FILTER vc._from == v._id
        RETURN 1
    )
    SORT cve_count DESC
    LIMIT 1
    RETURN {
        vendor_name: v.name,
        cve_count: cve_count
    }

AQL Result:
[{'vendor_name': 'microsoft', 'cve_count': 4965}]

> Finished chain.
Tools used: ['text_to_aql_to_text']
Multi-tool calling: False


## Vendor with the Most CVEs

Based on the analysis of the database, **Microsoft** has the highest number of CVEs (Common Vulnerabilities and Exposures) with **4,965** security vulnerabilities associated with them.

This significantly higher count compared to other vendors can be attributed to several factors:

1. Microsoft has an extensive product portfolio spanning operating systems, office software, cloud services, and more
2. Their products have widespread global adoption, making them a common target for security researchers and attackers
3. Microsoft has established mature vulnerability disclosure processes that encourage the reporting of security issues

It's worth noting that a higher CVE count doesn't necessarily indicate lower security quality, as it can also reflect more thorough security practices and transparency in vulnerability reporting.

In [ ]:
#Example query for Arango Query Language
query_graph("Which vendor has highest number of products?");

Using Arango Query Tool
Query: Which vendor has the highest number of products?


> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH vendor, product, product_vendor
FOR v IN vendor
    LET product_count = LENGTH(
        FOR p IN product
            FOR pv IN product_vendor
                FILTER pv._from == p._id AND pv._to == v._id
                RETURN p
    )
    SORT product_count DESC
    LIMIT 1
    RETURN {
        vendor_name: v.name,
        product_count: product_count
    }

AQL Result:
[{'vendor_name': 'hp', 'product_count': 2541}]

> Finished chain.
Tools used: ['text_to_aql_to_text']
Multi-tool calling: False


## Answer

Based on the database analysis, **HP** has the highest number of products with a total of **2,541 products** associated with them. This makes HP the vendor with the largest product portfolio in the database.

In [ ]:
#Example query for ArangoSearch (full-text search)
query_graph("How many CVE's have any of the following words attack, vulnerability, remote, server, authenticated, xss, csrf, injection in their summaries, list their count as a bar chart");

Using ArangoSearch Tool
Query: Count how many CVEs have each of these words in their summaries: attack, vulnerability, remote, server, authenticated, xss, csrf, injection. Return the counts for each keyword separately.
Generating query
ArangoSearch Query: LET keywords = ["attack", "vulnerability", "remote", "server", "authenticated", "xss", "csrf", "injection"]

LET counts = (
    FOR keyword IN keywords
        LET count = (
            FOR doc IN cve_search
                SEARCH ANALYZER(LIKE(doc.summary, CONCAT("%", keyword, "%")), "identity")
                COLLECT WITH COUNT INTO total
                RETURN total
        )[0]
        RETURN {
            keyword: keyword,
            count: count
        }
)

RETURN counts
Tools used: ['text_to_aql_arangosearch']
Multi-tool calling: False


Here are the counts of CVEs containing each keyword in their summaries, formatted as data for a bar chart:

| Keyword | Count |
|---------|-------|
| attack | 57,134 |
| vulnerability | 31,594 |
| remote | 48,028 |
| server | 6,616 |
| authenticated | 9,168 |
| xss | 21 |
| csrf | 17 |
| injection | 6,457 |

This data shows that "attack" appears most frequently in CVE summaries (57,134 occurrences), followed by "remote" (48,028) and "vulnerability" (31,594). The terms "xss" and "csrf" appear much less frequently, with only 21 and 17 occurrences respectively.

To create a bar chart from this data, you would plot the keywords on the x-axis and the counts on the y-axis. The large difference between the highest and lowest values suggests you might want to consider using a logarithmic scale for the y-axis to better visualize all the data points.

In [ ]:
#Example query for ArangoSearch (full-text search)
query_graph("What are the CVEs related to Cross-site Scripting (XSS) vulnerabilities in phpMyAdmin? Use BM25 to query the CVE descriptions for relevant matches related to XSS vulnerabilities in phpMyAdmin. View the BM25 scores to determine relevance");

Using ArangoSearch Tool
Query: Find CVEs related to Cross-site Scripting (XSS) vulnerabilities in phpMyAdmin. Show the BM25 scores to determine relevance.
Generating query
ArangoSearch Query: FOR doc IN cve_search
  SEARCH PHRASE(doc.summary, "Cross-site Scripting XSS phpMyAdmin", "text_en")
  OR ANALYZERS(TOKENS("Cross-site Scripting XSS", "text_en") ALL IN TOKENS(doc.summary, "text_en"), "text_en")
  AND PHRASE(doc.summary, "phpMyAdmin", "text_en")
  LET score = BM25(doc)
  SORT score DESC
  RETURN {
    cve_id: doc._key,
    summary: doc.summary,
    cvss: doc.cvss,
    cwe_code: doc.cwe_code,
    cwe_name: doc.cwe_name,
    pub_date: doc.pub_date,
    relevance_score: score
  }
Generating query
ArangoSearch Query: FOR doc IN cve_search
  SEARCH ANALYZER(
    doc.summary LIKE "cross-site scripting" OR 
    doc.summary LIKE "XSS" OR
    doc.cwe_name LIKE "cross-site scripting" OR
    doc.cwe_code LIKE "CWE-79"
  , "identity")
  AND ANALYZER(
    doc.summary LIKE "phpMyAdmin"
  , "ide

## XSS Vulnerabilities in phpMyAdmin: Analysis Based on BM25 Relevance Scores

Based on the BM25 search results, I've identified numerous Cross-site Scripting (XSS) vulnerabilities in phpMyAdmin across various versions. Here's an analysis of the most relevant findings:

### Most Relevant XSS Vulnerabilities (Highest BM25 Scores)

1. **CVE-2012-5368** (Score: 32.97)
   - Affects: phpMyAdmin 3.5.x before 3.5.3
   - Description: Uses JavaScript code obtained through an HTTP session to phpmyadmin.net without SSL, allowing man-in-the-middle attackers to conduct XSS attacks by modifying this code.

### Other Highly Relevant XSS Vulnerabilities (Score: 10.99)

2. **CVE-2011-2642**
   - Affects: phpMyAdmin before 3.3.10.3 and 3.4.x before 3.4.3.2
   - Description: Multiple XSS vulnerabilities in table Print view implementation in tbl_printview.php, allowing remote authenticated users to inject arbitrary web script via crafted table names.

3. **CVE-2011-3592**
   - Affects: phpMyAdmin 3.4.x before 3.4.5
   - Description: Multiple XSS vulnerabilities in PMA_unInlineEditRow function in js/sql.js, allowing injection via database names, table names, or column names.

4. **CVE-2014-5273**
   - Affects: phpMyAdmin 4.0.x before 4.0.10.2, 4.1.x before 4.1.14.3, and 4.2.x before 4.2.7.1
   - Description: Multiple XSS vulnerabilities across various pages including browse table, ENUM editor, monitor page, query charts, and table relations.

5. **CVE-2016-5733**
   - Affects: phpMyAdmin 4.0.x before 4.0.10.16, 4.4.x before 4.4.15.7, and 4.6.x before 4.6.3
   - Description: Multiple XSS vulnerabilities involving crafted table names, mysqld log_bin directives, transformations, AJAX error handling, Designer implementation, charts, and zoom-search.

### Common Attack Vectors

Based on the search results, the most common XSS attack vectors in phpMyAdmin include:

1. **Crafted table/database names** - Appears in numerous vulnerabilities
2. **Column names** - Often exploited in table views and editing interfaces
3. **SQL queries** - Malicious queries that trigger XSS when displayed
4. **Error messages** - Improper handling of error output
5. **JavaScript functions** - Particularly in AJAX operations and confirmation messages

### Affected Versions

The vulnerabilities span across multiple major versions of phpMyAdmin:
- 2.x series (older versions)
- 3.x series (particularly 3.4.x and 3.5.x)
- 4.x series (4.0.x, 4.1.x, 4.2.x, 4.4.x, 4.5.x, 4.6.x)

### Mitigation Patterns

Most of these vulnerabilities were addressed in point releases, suggesting that keeping phpMyAdmin updated is critical for security. Many of the fixes involved proper input validation and output encoding to prevent script injection.

This analysis shows that phpMyAdmin has had a significant history of XSS vulnerabilities, with the most critical ones involving crafted database and table names that can execute malicious JavaScript when displayed in the application interface.

In [ ]:
#Example query for networkX code generation
query_graph("Calculate modularity scores to understand the structural integrity of these vulnerability clusters");

Using networkX code generation tool

Generating NetworkX code
Query: Calculate modularity scores for vulnerability clusters in the graph to understand their structural integrity

Optimizing networkX code

2) Executing NetworkX code
Graph has 145506 nodes and 316954 edges
Starting modularity calculation...
Finding largest connected component...


[02:57:16 +0000] [INFO]: Graph 'CVE' load took 7.68485951423645s
[02:57:16 +0000] [INFO]: NXCG Graph construction took 0.402118444442749s


Error during analysis: Subgraphing an ArangoDB Graph is not yet implemented


NotImplementedError: Subgraphing an ArangoDB Graph is not yet implemented

EXEC ERROR: Execution error: Subgraphing an ArangoDB Graph is not yet implemented

Refactoring networkX code to resolve issues. Attempt:  1

2) Executing NetworkX code
Graph has 145506 nodes and 316954 edges
Starting modularity calculation...
Finding largest connected component...
Largest connected component has 81586 nodes and 228810 edges
Using fast greedy modularity for large graph...
Found 1650 communities in 27.67 seconds
Time limit approaching. Analyzed 4 communities so far.
Analysis completed in 55.11 seconds
Tools used: ['text_to_nx_algorithm_to_text']
Multi-tool calling: False


## Modularity Analysis of Vulnerability Clusters

I've analyzed the vulnerability clusters in the graph database to assess their structural integrity. Here are the findings:

### Overall Modularity Score
- **Global Modularity Score**: 0.874
- **Total Communities Detected**: 1,650

The overall modularity score of 0.874 indicates a very strong community structure within the vulnerability network. Scores above 0.7 generally suggest well-defined communities with clear boundaries.

### Top Vulnerability Clusters

| Community ID | Size | CVE Count | Local Modularity | Sample CVEs |
|--------------|------|-----------|------------------|-------------|
| 2 | 5,124 | 3,571 | 0.994 | CVE-2017-12284, CVE-2013-6976, CVE-2015-0588... |
| 1 | 5,329 | 4,922 | 0.991 | CVE-2013-5045, CVE-2010-0027, CVE-2016-3385... |
| 3 | 4,563 | 3,777 | 0.988 | CVE-2014-4789, CVE-2018-1461, CVE-2017-1428... |
| 0 | 6,387 | 5,739 | 0.809 | CVE-2017-2918, CVE-2015-5262, CVE-2018-18606... |

### Interpretation

1. **High Structural Integrity**: The top clusters all show extremely high local modularity scores (0.809-0.994), indicating that vulnerabilities within these clusters are densely connected to each other and sparsely connected to vulnerabilities in other clusters.

2. **Cluster Characteristics**:
   - Community 2: Has the highest local modularity (0.994) with 5,124 nodes and 3,571 CVEs
   - Community 1: Contains the second-highest number of CVEs (4,922) with very high modularity (0.991)
   - Community 0: Is the largest cluster with 6,387 nodes and 5,739 CVEs, but has relatively lower modularity (0.809)

3. **Security Implications**: These well-defined clusters suggest that vulnerabilities tend to group together based on common characteristics (possibly affecting similar systems, vendors, or vulnerability types). This clustering information can be valuable for prioritizing security responses and understanding vulnerability propagation patterns.

The high modularity scores across these clusters indicate that the vulnerability network has a strong community structure, which can help in developing targeted security strategies for specific vulnerability groups.

In [ ]:
#Example query for networkX code generation
query_graph("Identify the top 5 most central vendors/products that act as potential vulnerability spreaders");

Processing results...
Time to process results: 0.20 seconds
Total execution time: 215.79 seconds
Using networkX code generation tool

Generating NetworkX code
Query: Calculate degree centrality for all nodes and return the top 5 vendors or products with the highest scores, as these would represent nodes connected to many vulnerabilities

Optimizing networkX code

2) Executing NetworkX code
Graph has 145506 nodes and 316954 edges
Starting degree centrality calculation...
Found 16176 vendor nodes and 40554 product nodes
Calculating degree centrality...
Calculation completed in 181.66 seconds
Top 5 vendors/products by degree centrality:
1. Vendor: microsoft - Score: 0.036865
2. Vendor: cisco - Score: 0.035174
3. Vendor: ibm - Score: 0.031353
4. Vendor: google - Score: 0.028350
5. Vendor: apple - Score: 0.025978
Tools used: ['text_to_nx_algorithm_to_text', 'text_to_nx_algorithm_to_text', 'text_to_nx_algorithm_to_text']
Multi-tool calling: False


Based on the analysis using two different centrality measures, I can now provide you with the top 5 most central vendors/products that act as potential vulnerability spreaders:

## Top 5 Most Central Vendors/Products (Potential Vulnerability Spreaders)

### Based on Eigenvector Centrality
Eigenvector centrality measures influence in the network, with higher scores indicating nodes connected to other important nodes:

1. **debian** (Vendor) - 0.3347
2. **debian_linux** (Product) - 0.3281
3. **microsoft** (Vendor) - 0.2357
4. **redhat** (Vendor) - 0.2063
5. **canonical** (Vendor) - 0.1862

### Based on Degree Centrality
Degree centrality measures the number of direct connections, with higher scores indicating nodes connected to many other nodes:

1. **microsoft** (Vendor) - 0.0369
2. **cisco** (Vendor) - 0.0352
3. **ibm** (Vendor) - 0.0314
4. **google** (Vendor) - 0.0283
5. **apple** (Vendor) - 0.0260

### Key Insights:
- **Microsoft** appears in both lists, indicating its significance as a potential vulnerability spreader from both influence and connection perspectives.
- **Debian** and its product **debian_linux** have the highest eigenvector centrality, suggesting they are connected to other highly influential nodes in the vulnerability network.
- Major technology vendors like **Cisco**, **IBM**, **Google**, and **Apple** have high degree centrality, meaning they have many direct connections to vulnerabilities.

These vendors/products would be critical points for vulnerability management and security monitoring, as issues in these systems could potentially affect many other connected systems.

In [ ]:
#Example query for hybrid tool usage
query_graph("what are the top 5 products in terms of numbers of CVE's and which of them is integral to the network?");

Using Arango Query Tool
Query: Find the top 5 products with the highest number of CVEs


> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH product, product_cve, cve
FOR p IN product
    LET cve_count = LENGTH(
        FOR pc IN product_cve
        FILTER pc._from == p._id
        RETURN pc
    )
    SORT cve_count DESC
    LIMIT 5
    RETURN {
        product_name: p.name,
        cve_count: cve_count
    }

AQL Result:
[{'product_name': 'debian_linux', 'cve_count': 2902}, {'product_name': 'android', 'cve_count': 2348}, {'product_name': 'ubuntu_linux', 'cve_count': 1868}, {'product_name': 'linux_kernel', 'cve_count': 1764}, {'product_name': 'mac_os_x', 'cve_count': 1747}]

> Finished chain.
Using networkX code generation tool

Generating NetworkX code
Query: Calculate the betweenness centrality for the products: debian_linux, android, ubuntu_linux, linux_kernel, mac_os_x to determine which ones are most integral to the network

Optimizing networkX code

2) Executing Networ

NotImplementedError: Subgraphing an ArangoDB Graph is not yet implemented

EXEC ERROR: Execution error: Subgraphing an ArangoDB Graph is not yet implemented

Refactoring networkX code to resolve issues. Attempt:  1

2) Executing NetworkX code
Starting analysis...
Finding product nodes...
Found product: mac_os_x -> product/480761625
Found product: linux_kernel -> product/480761115
Found product: android -> product/480743898
Found product: ubuntu_linux -> product/480776551
Found product: debian_linux -> product/480750289
Calculating betweenness centrality on the full graph with sampling...
Calculating betweenness centrality...


[01:52:11 +0000] [INFO]: Graph 'CVE' load took 5.891673564910889s
[01:52:11 +0000] [INFO]: NXCG Graph construction took 0.42137861251831055s


Calculation completed in 8.03 seconds
Analysis completed successfully
Results: {'betweenness_centrality': {'debian_linux': 0.03461962193250656, 'linux_kernel': 0.009757774882018566, 'ubuntu_linux': 0.008693670853972435, 'android': 0.0031296154484152794, 'mac_os_x': 0.002414585091173649}, 'most_central_product': 'debian_linux', 'analysis': 'The product with the highest betweenness centrality is debian_linux, indicating it serves as the most important bridge in the vulnerability network.'}
Tools used: ['text_to_aql_to_text', 'text_to_nx_algorithm_to_text']
Multi-tool calling: True
Tool sequence: text_to_aql_to_text -> text_to_nx_algorithm_to_text


## Top 5 Products by CVE Count and Network Importance

Based on the analysis, here are the top 5 products with the highest number of CVEs:

1. **Debian Linux** - 2,902 vulnerabilities
2. **Android** - 2,348 vulnerabilities
3. **Ubuntu Linux** - 1,868 vulnerabilities
4. **Linux Kernel** - 1,764 vulnerabilities
5. **Mac OS X** - 1,747 vulnerabilities

### Network Importance Analysis

When examining which of these products is most integral to the network, I analyzed their betweenness centrality, which measures how often a node appears on the shortest paths between other nodes in the network. Higher betweenness centrality indicates that a product serves as an important bridge or connector in the vulnerability network.

The betweenness centrality scores are:
- **Debian Linux**: 0.03462
- **Linux Kernel**: 0.00976
- **Ubuntu Linux**: 0.00869
- **Android**: 0.00313
- **Mac OS X**: 0.00241

**Debian Linux** is clearly the most integral product to the network, with a betweenness centrality significantly higher than the other top products. This suggests that Debian Linux not only has the highest number of vulnerabilities but also serves as a critical connection point in the vulnerability ecosystem, potentially linking various other products, vendors, and vulnerability types.

This could be due to Debian's widespread use as a base for many other distributions, its extensive package repository, or its role in the broader software ecosystem.

In [ ]:
#Example query for hybrid tool usage
query_graph("what is the influence of the word 'xss' on the CVE's and draw the results as a directed graph.");

ERROR! Session/line number was not unique in database. History logging moved to new session 31
Using ArangoSearch Tool
Query: Find all CVEs that mention 'xss' in their description or details
Generating query
ArangoSearch Query: FOR doc IN cve_search
  SEARCH ANALYZER(doc.summary LIKE '%xss%', "identity")
  RETURN {
    _id: doc._id,
    _key: doc._key,
    summary: doc.summary,
    pub_date: doc.pub_date,
    cvss: doc.cvss,
    cwe_code: doc.cwe_code,
    cwe_name: doc.cwe_name
  }
Using Arango Query Tool
Query: Find the relationships between CVEs containing 'xss' in their summary and other entities like vendors, products, and CWEs. Show how these XSS vulnerabilities are connected in the graph.


> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH cve, product, vendor, product_cve, product_vendor, vendor_cve
FOR v IN cve
    FILTER CONTAINS(LOWER(v.summary), 'xss')
    LET related_products = (
        FOR p, e IN INBOUND v product_cve
            RETURN {
                pr

AttributeError: 'Graph' object has no attribute 'predecessors'

EXEC ERROR: Execution error: 'Graph' object has no attribute 'predecessors'

Refactoring networkX code to resolve issues. Attempt:  1

2) Executing NetworkX code
Starting XSS vulnerability graph creation...
Filtering CVEs with 'xss' in summary...
Found 12252 CVEs with XSS references
Limiting analysis to 50 CVEs out of 12252
Finding products and vendors connected to XSS CVEs...
Error during XSS vulnerability graph creation: 'Graph' object has no attribute 'in_edges'
Error in XSS vulnerability analysis: 'Graph' object has no attribute 'in_edges'


AttributeError: 'Graph' object has no attribute 'in_edges'

EXEC ERROR: Execution error: 'Graph' object has no attribute 'in_edges'

Refactoring networkX code to resolve issues. Attempt:  2

2) Executing NetworkX code
Starting XSS vulnerability graph creation...
Filtering CVEs with 'xss' in summary...
Found 12252 CVEs with XSS references
Limiting analysis to 50 CVEs out of 12252
Finding products and vendors connected to XSS CVEs...
Approaching time limit, stopping early
Created graph with 12282 nodes and 54 edges
Analysis completed in 60.08 seconds
Tools used: ['text_to_aql_arangosearch', 'text_to_aql_to_text', 'text_to_nx_algorithm_to_text']
Multi-tool calling: True
Tool sequence: text_to_aql_arangosearch -> text_to_aql_to_text -> text_to_nx_algorithm_to_text


Based on the data I've gathered, I'll now prepare the information for a directed graph visualization of the influence of 'xss' on CVEs.

## XSS Influence on CVEs - Directed Graph Data

The data for your directed graph visualization shows the following key relationships:

### Node Types:
1. **CVEs** - 50 vulnerabilities containing 'xss' in their descriptions
2. **CWEs** - 2 primary weakness types:
   - CWE-79: Improper Neutralization of Input During Web Page Generation (Cross-site Scripting)
   - CWE-352: Cross-Site Request Forgery (CSRF)
3. **Vendors** - 14 affected vendors including Cisco, D-Link, VMware, Drupal, and others
4. **Products** - 14 affected products

### Key Relationships:
1. **CVE → CWE**: Most XSS-related CVEs (48) are classified as CWE-79, while 2 are classified as CWE-352
2. **Vendor → Product**: 17 connections showing which vendors own which products
3. **Product → CVE**: 14 direct connections between products and specific XSS vulnerabilities

### Graph Structure:
- The graph is centered around CWE-79 (Cross-site Scripting), which connects to most of the CVEs
- Vendors connect to their products, which in turn connect to specific CVEs
- The severity of vulnerabilities (CVSS scores) ranges from 1.5 to 7.5, with most falling in the 3.5-4.3 range

### Directed Graph Visualization Instructions:
1. **Node Colors**:
   - CVEs: Red (with intensity based on CVSS score)
   - CWEs: Orange
   - Vendors: Blue
   - Products: Green

2. **Edge Directions**:
   - CWE → CVE (showing which weakness type each vulnerability belongs to)
   - Vendor → Product (showing ownership)
   - Product → CVE (showing which product is affected by which vulnerability)

3. **Node Sizes**:
   - CWE nodes should be larger, proportional to the number of connected CVEs
   - Vendor nodes sized by number of affected products
   - CVE nodes can be uniform size or varied by CVSS score

This directed graph will effectively visualize how XSS vulnerabilities affect various products across different vendors, and how they relate to specific weakness types in the CWE classification system.

In [ ]:
!pip install flask pyngrok

In [ ]:
# # Making the server publicly accessible using static domain
# # The returned URL is passed as env variable for claude MCP

from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/process', methods=['POST'])
def process():
    data = request.json
    query = data.get("query", "")
    result = query_graph(query)
    return jsonify(result)

#uncomment any of the below lines to make the notebook publicly accessible, this url should be provided as env variable to claude MCP. Detailed usage for it can it found in mcp-server folder

# Start ngrok tunnel with static domain
#public_url = ngrok.connect(5000, hostname="your-static-host")

#start the ngrok with random domain
#public_url = ngrok.connect(5000)

#print(f"Public URL: {public_url}")

# Run Flask server without reloader to prevent blocking
app.run(port=5000, use_reloader=False)
